<a href="https://colab.research.google.com/github/dsogden/IBM_tools_assigment/blob/main/TeslavsGamestop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Define the graphing function

In [2]:
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = 0.3)
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date, infer_datetime_format=True), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date, infer_datetime_format=True), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    fig.show()

## Extract Stock Data

In [3]:
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period='max')

In [4]:
tesla_data.reset_index(inplace=True)
tesla_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Webscrape the stock data

In [5]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
html_data = requests.get(url).text

In [19]:
tesla_revenue = pd.read_html(url)[1]

In [20]:
columns = tesla_revenue.columns
replace = ['Date', 'Quarterly_Revenue']
rename_columns = {current: replace[index] for index, current in enumerate(columns)}
tesla_revenue = tesla_revenue.rename(columns=rename_columns)

In [8]:
tesla_revenue = tesla_revenue.dropna()
tesla_revenue.tail(5)

,Date,Quarterly_Revenue
48,2010-09-30,$31
49,2010-06-30,$28
50,2010-03-31,$21
52,2009-09-30,$46
53,2009-06-30,$27


### Define a function that will return the stock ticker data

In [9]:
def stock_data_retrive(symbol, period):
    stock = yf.Ticker(symbol)
    history = stock.history(period=period)
    history = history.reset_index()
    return history

In [21]:
gme_data = stock_data_retrive('GME', 'max')
gme_data.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


### Function that will webscrape

In [11]:
def scrape_data(url):
    data = pd.read_html(url)[1]
    columns = data.columns
    replace = ['Date', 'Revenue']
    rename_columns = {current: replace[index] for index, current in enumerate(columns)}
    data = data.rename(columns=rename_columns)
    data = data.dropna()
    return data

In [22]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html'
gme_revenue = scrape_data(url)
gme_revenue.tail(5)

,Date,Revenue
57,2006-01-31,"$1,667"
58,2005-10-31,$534
59,2005-07-31,$416
60,2005-04-30,$475
61,2005-01-31,$709


In [13]:
gme_revenue.tail(5)

,Date,Revenue
57,2006-01-31,"$1,667"
58,2005-10-31,$534
59,2005-07-31,$416
60,2005-04-30,$475
61,2005-01-31,$709


In [14]:
gme_revenue['Revenue'] = gme_revenue['Revenue'].apply(lambda x: x.split('$')[-1])

In [15]:
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace(',', '')

In [16]:
def update_revenue(data, column):
    data[column] = data[column].apply(lambda x: x.split('$')[-1])
    data[column] = data[column].str.replace(',', '')
    return data

## Plot Tesla stock data

In [17]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm'
tesla_revenue = scrape_data(url)
tesla_revenue = update_revenue(tesla_revenue, 'Revenue')
make_graph(tesla_data, tesla_revenue, 'Tesla')

In [18]:
make_graph(gme_data, gme_revenue, 'GameStop')